In [1]:
import os
import random
import numpy as np
import zarr
import ezomero
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, add_metadata

from biohack_utils.CollectionModel import (
    OMECollection, OMEWrapper, MultiscaleNode, 
    NodeAttributes, OMEROInfo, LabelInfo
)
from biohack_utils.model_utils import get_collection_as_model

In [2]:
conn = ezomero.connect(
    os.getenv("OMERO_USERNAME"), 
    os.getenv("OMERO_PASSWORD"), 
    os.getenv("OMERO_GROUP"),
    host=os.getenv("OMERO_HOST"),
    port=int(os.getenv("OMERO_PORT")),
    secure=os.getenv("OMERO_SECURE") == "True"
)

In [6]:
collection_ann_id = 11022

In [7]:
collection = get_collection_as_model(conn, collection_ann_id)

# Model gives us structured access
intensity_nodes = collection.get_intensity_nodes()
label_nodes = collection.get_label_nodes()

print("Intensities:", [n.name for n in intensity_nodes])
print("Labels:", [n.name for n in label_nodes])


Intensities: ['Raw']
Labels: ['Nuclei_segmentation', 'Cell_segmentation']


In [8]:
def fetch_image(conn, node: MultiscaleNode) -> np.ndarray:
    """Fetch image array for a node."""
    image_id = node.attributes.omero.image_id
    _, arr = ezomero.get_image(conn, image_id)
    return arr

intensity_data = {n.name: fetch_image(conn, n) for n in intensity_nodes}
label_data = {n.name: fetch_image(conn, n) for n in label_nodes}

In [14]:
zarr_path = "{}.ome.zarr".format(collection.name)
if os.path.exists(zarr_path):
    raise RuntimeError("{} already exists".format(zarr_path))

store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

In [27]:
def write_intensity_image(root, node: MultiscaleNode, data: np.ndarray):
    """Write an intensity image and update node path."""
    data = data.squeeze().astype(np.uint16)
    if data.ndim == 3:
        data = np.moveaxis(data, -1, 0)  # TZCYX -> CYX
    
    axes = "cyx" if data.ndim == 3 else "yx"
    chunks = data.shape  # simple chunking
    
    grp = root.create_group(node.name)
    write_image(image=data, group=grp, axes=axes, 
                storage_options=dict(chunks=chunks))
    
    # Add rendering metadata
    add_metadata(grp, {
        "omero": {
            "channels": [{
                "color": "FFFFFF",
                "window": {"start": int(data.min()), "end": int(data.max()),
                          "min": int(data.min()), "max": int(data.max())},
                "label": node.name,
                "active": True,
            }]
        }
    })
    
    # Update node path for collection metadata
    node.path = "./{}".format(node.name)
    return grp

def write_label_image(root, intensity_grp, node: MultiscaleNode, 
                      data: np.ndarray, axes: str, chunks: tuple):
    """Write a label image under its source intensity image."""
    data = data.squeeze().astype(np.int32)
    if data.ndim == 2:
        data = np.moveaxis(data, -1, 0)
    
    # Create labels group if needed
    if "labels" not in intensity_grp:
        labels_root = intensity_grp.create_group("labels")
        add_metadata(labels_root, {"labels": []})
    else:
        labels_root = intensity_grp["labels"]
    
    # Update labels list
    current_labels = labels_root.attrs.get("ome", {}).get("labels", [])
    current_labels.append(node.name)
    add_metadata(labels_root, {"labels": current_labels})
    
    # Write label data
    label_grp = labels_root.create_group(node.name)
    write_image(image=data, group=label_grp, axes=axes,
                storage_options=dict(chunks=chunks))
    
    # Generate colors
    colors_meta = generate_label_colors(data)
    add_metadata(label_grp, {"image-label": {"colors": colors_meta}})
    
    # Update node path
    source_name = node.get_sources()[0] if node.get_sources() else "unknown"
    node.path = "./{}/labels/{}".format(source_name, node.name)

def generate_label_colors(data: np.ndarray) -> list:
    """Generate random colors for label values."""
    random.seed(12345)
    unique_labels = np.unique(data)
    colors = []
    for lv in unique_labels:
        lv = int(lv)
        if lv == 0:
            colors.append({"label-value": lv, "rgba": [0, 0, 0, 0]})
        else:
            colors.append({
                "label-value": lv,
                "rgba": [random.randint(80, 255) for _ in range(3)] + [255]
            })
    return colors

In [25]:
# Write intensity images
intensity_groups = {}
for node in intensity_nodes:
    data = intensity_data[node.name]
    grp = write_intensity_image(root, node, data)
    intensity_groups[node.name] = grp


In [22]:
grp

<Group file://image.ome.zarr/Raw>

In [33]:
# Write label images - use first intensity image as parent
if not intensity_nodes:
    print("Warning: no intensity images, cannot write labels")
else:
    default_source = intensity_nodes[0]
    default_grp = intensity_groups[default_source.name]
    default_data = intensity_data[default_source.name]
    
    axes = "yx" 
    chunks = data.squeeze().shape
    
    for node in label_nodes:
        data = label_data[node.name]
        write_label_image(root, default_grp, node, data, axes, chunks)

In [34]:
wrapper = OMEWrapper(ome=collection)
root.attrs["ome"] = wrapper.ome.model_dump(exclude_none=True)

print("Wrote OME-Zarr to: {}".format(os.path.abspath(zarr_path)))

Wrote OME-Zarr to: /var/home/maartenpaul/Documents/GitHub/BioHackatonDE_annotations/image.ome.zarr
